## Chapter 1.2: Data Structure and Manipulation
***
In the previous tutorial we saw how we can create a model by calling a few lines of code from imported data. However modelling tends to be an iterative process. Here we will explore the tools that `GemPy` with the help of `pandas` offers to modify the input data of a model.

There is 5 main  funtion "types" in GemPy:

- *create*: 
    - create new objects
    - return the objects
 
- *set*
    - set given values **inplace**

- *update*
    - update dataframe or other attribute from other object or many objects. Usually this object is not passed as argument (this is the main difference with map)

- *map*
    - update dataframe (so far mainly df) or other attribute from an object to another object.
    - Completelly directed. One attribute/property is updated by another one.
    - In general, we map_from so the method is in the mutated object.

- *get*
    - return an image of the object
    
The intention is that a function/method that does not fall in any of these categories has a name (verb in principle) self explanatory. 


As always we start importing the usual packages and reading expample data:

In [1]:
# These two lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../..")

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import pandas as pn
import matplotlib.pyplot as plt


## Series

Series is the object that contains the properties associated with each independent scalar field. Right now it is simply the order of the series (which is infered by the index order). But in the future will be add the unconformity relation or perhaps the type of interpolator

Series and Faults classes are quite entagled since fauls are a type of series

In [2]:
faults = gp.Faults()
series = gp.Series(faults)
series.df

,order_series,BottomRelation
Default series,1,Erosion


We can modify the series bt using `set_series_index`:

In [3]:
series.set_series_index(['foo', 'foo2', 'foo5', 'foo7'])
series

,order_series,BottomRelation
foo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


The index of series are pandas categories. These provides quite handy backend functionality (see pandas.Categorical).

In [4]:
series.df.index

CategoricalIndex(['foo', 'foo2', 'foo5', 'foo7'], categories=['foo', 'foo2', 'foo5', 'foo7'], ordered=False, dtype='category')

For adding new series:

In [5]:
series.add_series('foo3')
series

,order_series,BottomRelation
foo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion
foo3,5,Erosion


Delete series

In [6]:
series.delete_series('foo3')
series

,order_series,BottomRelation
foo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


Rename series:

In [7]:
series.rename_series({'foo':'boo'})
series

,order_series,BottomRelation
boo,1,Erosion
foo2,2,Erosion
foo5,3,Erosion
foo7,4,Erosion


Reorder series:

In [8]:
series.reorder_series(['foo2', 'boo', 'foo7', 'foo5'])
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion


### Faults

The *df faults* is used to charectirize which *mathematical series* behave as fault and if mentioned faults are finite or infinite. Both df should get updated automatically as we modify the series object linked to the fault object (by passing it wehn a Series object is created).

In [9]:
faults

,isFault,isFinite
foo2,False,False
boo,False,False
foo7,False,False
foo5,False,False


Finally we have the *faults relations df* which captures which *mathematical series* a given fault offset in order to reproduce complex faulting networks

In [10]:
faults.faults_relations_df

,foo2,boo,foo7,foo5
foo2,False,False,False,False
boo,False,False,False,False
foo7,False,False,False,False
foo5,False,False,False,False


We can use `set_is_fault` to choose which of our series are faults:

In [11]:
faults.set_is_fault(['boo'])

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False


Similar thing for the fault relations:

In [12]:
fr = np.zeros((4, 4))
fr[2, 2] = True
faults.set_fault_relation(fr)

,foo2,boo,foo7,foo5
foo2,False,False,False,False
boo,False,False,False,False
foo7,False,False,False,False
foo5,False,False,False,False


Now if we change the series df and we update the series already defined will conserve their values while the new ones will be set to false:

In [13]:
series.add_series('foo20')


In [14]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


In [15]:
faults

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


In [16]:
faults.faults_relations_df

,foo2,boo,foo7,foo5,foo20
foo2,False,False,False,False,False
boo,False,False,False,False,False
foo7,False,False,False,False,False
foo5,False,False,False,False,False
foo20,False,False,False,False,False


When we add new series the values switch  to NaN. We will be careful not having any nan in the DataFrames or we will raise errors down the line.

In [17]:
faults.set_is_fault()

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


In [18]:
faults.set_fault_relation()

,foo2,boo,foo7,foo5,foo20
foo2,False,False,False,False,False
boo,False,False,False,False,False
foo7,False,False,False,False,False
foo5,False,False,False,False,False
foo20,False,False,False,False,False


### Formations:

The *df* formation contain three properties. *id* refers to the order of the formation on the sequential pile, i.e. the strict order of computation. *values* on the other hand is the final value that each voxel will have after discretization. This may be useful for example in the case we want to map a specific geophysical property (such as density) to a given unity. By default both are the same since to discretize lithological units the value is arbitrary.

#### From an empty df

The Formation class needs to have an associate series object. This will limit the name of the series since they are a pandas.Category

In [19]:
surfaces = gp.Surfaces(series)

We can set any number of formations by passing a list with the names. By default they will take the name or the first series.

In [20]:
surfaces.set_surfaces_names(['foo', 'foo2', 'foo5'])


True

In [21]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


We can add new formations:

In [22]:
surfaces.add_surface(['feeeee'])
surfaces

,surface,series,order_surfaces,isBasement,color,id
0,foo,foo20,1,False,#015482,1
1,foo2,foo20,2,False,#9f0052,2
2,foo5,foo20,3,False,#ffbe00,3
3,feeeee,foo20,4,True,#728f02,4


The column formation is also a pandas.Categories. This will be important for the Data clases (surface_points and Orientations)

In [23]:
surfaces.df['surface']

0       foo
1      foo2
2      foo5
3    feeeee
Name: surface, dtype: object

In [24]:
surfaces

,surface,series,order_surfaces,isBasement,color,id
0,foo,foo20,1,False,#015482,1
1,foo2,foo20,2,False,#9f0052,2
2,foo5,foo20,3,False,#ffbe00,3
3,feeeee,foo20,4,True,#728f02,4


In [25]:
surfaces

,surface,series,order_surfaces,isBasement,color,id
0,foo,foo20,1,False,#015482,1
1,foo2,foo20,2,False,#9f0052,2
2,foo5,foo20,3,False,#ffbe00,3
3,feeeee,foo20,4,True,#728f02,4


### Set values

To set the values we do it with the following method

In [26]:
surfaces.set_surfaces_values([2,2,2,5])

,surface,series,order_surfaces,isBasement,color,id,value_0
0,foo,foo20,1,False,#015482,1,2
1,foo2,foo20,2,False,#9f0052,2,2
2,foo5,foo20,3,False,#ffbe00,3,2
3,feeeee,foo20,4,True,#728f02,4,5


In [27]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,value_0
0,foo,foo20,1,False,#015482,1,2
1,foo2,foo20,2,False,#9f0052,2,2
2,foo5,foo20,3,False,#ffbe00,3,2
3,feeeee,foo20,4,True,#728f02,4,5


#### Set values with a given name:

We can give specific names to the properties (i.e. density)

In [28]:
surfaces.add_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])

,surface,series,order_surfaces,isBasement,color,id,value_0,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,2,1
3,feeeee,foo20,4,True,#728f02,4,5,6,8


In [29]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,value_0,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,2,1
3,feeeee,foo20,4,True,#728f02,4,5,6,8


### Delete formations values

To delete a full propery:

In [30]:
surfaces.delete_surface_values(['val_foo', 'value_0'])

True

#### One of the formations must be set be the basement:

In [31]:
surfaces.set_basement()
surfaces

,surface,series,order_surfaces,isBasement,color,id,val2_foo
0,foo,foo20,1,False,#015482,1,2
1,foo2,foo20,2,False,#9f0052,2,2
2,foo5,foo20,3,False,#ffbe00,3,1
3,feeeee,foo20,4,True,#728f02,4,8


#### Set formation values

We can also use set values instead adding. This will delete the previous properties and add the new one

In [32]:
surfaces.set_surfaces_values([[2,2,2,6], [2,2,1,8]], ['val_foo', 'val2_foo'])
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,1
3,feeeee,foo20,4,True,#728f02,4,6,8


The last property is the correspondant series that each formation belong to. `series` and `formation` are pandas categories. To get a overview of what this mean check https://pandas.pydata.org/pandas-docs/stable/categorical.html.

In [33]:
surfaces.df['series']

0    foo20
1    foo20
2    foo20
3    foo20
Name: series, dtype: category
Categories (5, object): [foo2, boo, foo7, foo5, foo20]

In [34]:
surfaces.df['surface']

0       foo
1      foo2
2      foo5
3    feeeee
Name: surface, dtype: object

### Map series to formation

To map a series to a formation we can do it by passing a dict:

In [35]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
0,foo,foo20,1,False,#015482,1,2,2
1,foo2,foo20,2,False,#9f0052,2,2,2
2,foo5,foo20,3,False,#ffbe00,3,2,1
3,feeeee,foo20,4,True,#728f02,4,6,8


In [36]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


If a series does not exist in the `Series` object, we rise a warning and we set those formations to nans

In [37]:
d =  {"foo7":'foo', "booX": ('foo2','foo5', 'fee')}

In [38]:
surfaces.map_series(d)

In [39]:
surfaces.map_series({"foo7":'foo', "boo": ('foo2','foo5', 'fee')})

In [40]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,foo2,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,2,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


An advantage of categories is that they are order so no we can tidy the df by series and formation

### Modify formation name

In [41]:
surfaces.rename_surfaces({'foo2':'lala'})

True

In [42]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,lala,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,2,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


In [43]:
surfaces.df.loc[2, 'val_foo'] = 22

In [44]:
surfaces

,surface,series,order_surfaces,isBasement,color,id,val_foo,val2_foo
1,lala,boo,1,False,#9f0052,1,2,2
2,foo5,boo,2,False,#ffbe00,2,22,1
0,foo,foo7,1,False,#015482,3,2,2
3,feeeee,foo20,1,True,#728f02,4,6,8


# Data
#### surface_points
These two DataFrames (df from now on) will contain the individual information of each point at an interface or orientation. Some properties of this table are mapped from the *df* below.

In [47]:
surface_points = gp.SurfacePoints(surfaces)
#orientations = gp.Orientations()

In [48]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series


In [49]:
surface_points.set_surface_points(pn.DataFrame(np.random.rand(6,3)), ['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [50]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.989980,0.199619,0.131031,NaN,NaN,NaN,foo,NaN,NaN,1
1,0.733557,0.710490,0.657090,NaN,NaN,NaN,foo5,NaN,NaN,1
2,0.059345,0.010588,0.681098,NaN,NaN,NaN,lala,NaN,NaN,1
3,0.696453,0.191715,0.119883,NaN,NaN,NaN,foo5,NaN,NaN,1
4,0.404212,0.451895,0.428327,NaN,NaN,NaN,lala,NaN,NaN,1
5,0.235594,0.583180,0.299830,NaN,NaN,NaN,feeeee,NaN,NaN,1


In [51]:
surface_points.map_data_from_surfaces(surfaces, 'series')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.989980,0.199619,0.131031,NaN,NaN,NaN,foo,foo7,NaN,1
1,0.733557,0.710490,0.657090,NaN,NaN,NaN,foo5,boo,NaN,1
2,0.059345,0.010588,0.681098,NaN,NaN,NaN,lala,boo,NaN,1
3,0.696453,0.191715,0.119883,NaN,NaN,NaN,foo5,boo,NaN,1
4,0.404212,0.451895,0.428327,NaN,NaN,NaN,lala,boo,NaN,1
5,0.235594,0.583180,0.299830,NaN,NaN,NaN,feeeee,foo20,NaN,1


In [52]:
surface_points.map_data_from_surfaces(surfaces, 'id')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.989980,0.199619,0.131031,NaN,NaN,NaN,foo,foo7,3,1
1,0.733557,0.710490,0.657090,NaN,NaN,NaN,foo5,boo,2,1
2,0.059345,0.010588,0.681098,NaN,NaN,NaN,lala,boo,1,1
3,0.696453,0.191715,0.119883,NaN,NaN,NaN,foo5,boo,2,1
4,0.404212,0.451895,0.428327,NaN,NaN,NaN,lala,boo,1,1
5,0.235594,0.583180,0.299830,NaN,NaN,NaN,feeeee,foo20,4,1


In [53]:
series

,order_series,BottomRelation
foo2,1,Erosion
boo,2,Erosion
foo7,3,Erosion
foo5,4,Erosion
foo20,5,Erosion


In [54]:
surface_points.map_data_from_series(series, 'order_series')
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
0,0.989980,0.199619,0.131031,NaN,NaN,NaN,foo,foo7,3,3
1,0.733557,0.710490,0.657090,NaN,NaN,NaN,foo5,boo,2,2
2,0.059345,0.010588,0.681098,NaN,NaN,NaN,lala,boo,1,2
3,0.696453,0.191715,0.119883,NaN,NaN,NaN,foo5,boo,2,2
4,0.404212,0.451895,0.428327,NaN,NaN,NaN,lala,boo,1,2
5,0.235594,0.583180,0.299830,NaN,NaN,NaN,feeeee,foo20,4,5


In [55]:
surface_points.sort_table()
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
2,0.059345,0.010588,0.681098,NaN,NaN,NaN,lala,boo,1,2
4,0.404212,0.451895,0.428327,NaN,NaN,NaN,lala,boo,1,2
1,0.733557,0.710490,0.657090,NaN,NaN,NaN,foo5,boo,2,2
3,0.696453,0.191715,0.119883,NaN,NaN,NaN,foo5,boo,2,2
0,0.989980,0.199619,0.131031,NaN,NaN,NaN,foo,foo7,3,3
5,0.235594,0.583180,0.299830,NaN,NaN,NaN,feeeee,foo20,4,5


In [56]:
faults

,isFault,isFinite
foo2,False,False
boo,True,False
foo7,False,False
foo5,False,False
foo20,False,False


### Orientations

In [57]:
orientations = gp.Orientations(surfaces)

In [58]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series


### Set values passing pole vectors:

In [59]:
orientations.set_orientations(np.random.rand(6,3)*10,
                            np.random.rand(6,3),
                            surface=['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [60]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,6.754686,5.690667,1.158848,NaN,NaN,NaN,0.821686,0.107580,0.253786,75.298341,82.540911,1,foo,NaN,NaN,1
1,6.374983,4.590934,3.348953,NaN,NaN,NaN,0.418877,0.406944,0.134333,82.279939,45.827846,1,foo5,NaN,NaN,1
2,3.145834,6.076850,9.363157,NaN,NaN,NaN,0.279984,0.284805,0.175311,79.903256,44.511000,1,lala,NaN,NaN,1
3,1.472914,7.915580,0.675355,NaN,NaN,NaN,0.816654,0.023639,0.724555,43.568158,88.341968,1,foo5,NaN,NaN,1
4,8.989578,9.016743,5.295090,NaN,NaN,NaN,0.571805,0.610214,0.650213,49.442320,43.138833,1,lala,NaN,NaN,1
5,1.879135,7.531495,4.821511,NaN,NaN,NaN,0.222093,0.883543,0.888190,27.353344,14.109922,1,feeeee,NaN,NaN,1


### Set values pasing orientation data: azimuth, dip, pole (dip direction)

In [61]:
orientations.set_orientations(np.random.rand(6,3)*10,
                            orientation = np.random.rand(6,3)*20,
                            surface=['foo', 'foo5', 'lala', 'foo5', 'lala', 'feeeee'])

In [62]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,2.236362,9.289332,7.069177,NaN,NaN,NaN,0.009900,0.064134,0.239981,15.131408,8.775031,0.248601,foo,NaN,NaN,1
1,3.746931,5.376924,3.602518,NaN,NaN,NaN,1.293404,3.921952,12.104145,18.838738,18.251817,12.789250,foo5,NaN,NaN,1
2,1.009549,0.824145,7.769260,NaN,NaN,NaN,0.635225,3.700102,13.204787,15.870903,9.741438,13.728097,lala,NaN,NaN,1
3,7.664131,0.616635,7.039326,NaN,NaN,NaN,0.034972,0.271297,10.147020,1.544194,7.345296,10.150706,foo5,NaN,NaN,1
4,4.119740,2.983007,6.754145,NaN,NaN,NaN,0.019351,0.621533,2.959713,11.865248,1.783266,3.024331,lala,NaN,NaN,1
5,7.300673,3.089303,9.703372,NaN,NaN,NaN,0.240818,0.719070,5.363631,8.047284,18.515804,5.416973,feeeee,NaN,NaN,1


### Mapping data from the other df

In [63]:
orientations.map_data_from_surfaces(surfaces, 'series')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,2.236362,9.289332,7.069177,NaN,NaN,NaN,0.009900,0.064134,0.239981,15.131408,8.775031,0.248601,foo,foo7,NaN,1
1,3.746931,5.376924,3.602518,NaN,NaN,NaN,1.293404,3.921952,12.104145,18.838738,18.251817,12.789250,foo5,boo,NaN,1
2,1.009549,0.824145,7.769260,NaN,NaN,NaN,0.635225,3.700102,13.204787,15.870903,9.741438,13.728097,lala,boo,NaN,1
3,7.664131,0.616635,7.039326,NaN,NaN,NaN,0.034972,0.271297,10.147020,1.544194,7.345296,10.150706,foo5,boo,NaN,1
4,4.119740,2.983007,6.754145,NaN,NaN,NaN,0.019351,0.621533,2.959713,11.865248,1.783266,3.024331,lala,boo,NaN,1
5,7.300673,3.089303,9.703372,NaN,NaN,NaN,0.240818,0.719070,5.363631,8.047284,18.515804,5.416973,feeeee,foo20,NaN,1


In [64]:
orientations.map_data_from_surfaces(surfaces, 'id')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,2.236362,9.289332,7.069177,NaN,NaN,NaN,0.009900,0.064134,0.239981,15.131408,8.775031,0.248601,foo,foo7,3,1
1,3.746931,5.376924,3.602518,NaN,NaN,NaN,1.293404,3.921952,12.104145,18.838738,18.251817,12.789250,foo5,boo,2,1
2,1.009549,0.824145,7.769260,NaN,NaN,NaN,0.635225,3.700102,13.204787,15.870903,9.741438,13.728097,lala,boo,1,1
3,7.664131,0.616635,7.039326,NaN,NaN,NaN,0.034972,0.271297,10.147020,1.544194,7.345296,10.150706,foo5,boo,2,1
4,4.119740,2.983007,6.754145,NaN,NaN,NaN,0.019351,0.621533,2.959713,11.865248,1.783266,3.024331,lala,boo,1,1
5,7.300673,3.089303,9.703372,NaN,NaN,NaN,0.240818,0.719070,5.363631,8.047284,18.515804,5.416973,feeeee,foo20,4,1


In [65]:
orientations.map_data_from_series(series, 'order_series')
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series
0,2.236362,9.289332,7.069177,NaN,NaN,NaN,0.009900,0.064134,0.239981,15.131408,8.775031,0.248601,foo,foo7,3,3
1,3.746931,5.376924,3.602518,NaN,NaN,NaN,1.293404,3.921952,12.104145,18.838738,18.251817,12.789250,foo5,boo,2,2
2,1.009549,0.824145,7.769260,NaN,NaN,NaN,0.635225,3.700102,13.204787,15.870903,9.741438,13.728097,lala,boo,1,2
3,7.664131,0.616635,7.039326,NaN,NaN,NaN,0.034972,0.271297,10.147020,1.544194,7.345296,10.150706,foo5,boo,2,2
4,4.119740,2.983007,6.754145,NaN,NaN,NaN,0.019351,0.621533,2.959713,11.865248,1.783266,3.024331,lala,boo,1,2
5,7.300673,3.089303,9.703372,NaN,NaN,NaN,0.240818,0.719070,5.363631,8.047284,18.515804,5.416973,feeeee,foo20,4,5


In [66]:
orientations.update_annotations()

### Grid

In [67]:
grid = gp.Grid()
grid.set_regular_grid([0,10,0,10,0,10], [50,50,50])

In [68]:
grid.values

array([[0.1, 0.1, 0.1],
       [0.1, 0.1, 0.3],
       [0.1, 0.1, 0.5],
       ...,
       [9.9, 9.9, 9.5],
       [9.9, 9.9, 9.7],
       [9.9, 9.9, 9.9]])

#### Rescaling Data

In [69]:
rescaling = gp.RescaledData(surface_points, orientations, grid)


In [70]:
surface_points

,X,Y,Z,X_r,Y_r,Z_r,surface,series,id,order_series
2,0.059345,0.010588,0.681098,0.301718,0.258050,0.279380,lala,boo,1,2
4,0.404212,0.451895,0.428327,0.319710,0.281074,0.266192,lala,boo,1,2
1,0.733557,0.710490,0.657090,0.336893,0.294566,0.278128,foo5,boo,2,2
3,0.696453,0.191715,0.119883,0.334957,0.267500,0.250100,foo5,boo,2,2
0,0.989980,0.199619,0.131031,0.350272,0.267912,0.250682,foo,foo7,3,3
5,0.235594,0.583180,0.299830,0.310913,0.287924,0.259488,feeeee,foo20,4,5


In [71]:
orientations

,X,Y,Z,X_r,Y_r,Z_r,G_x,G_y,G_z,dip,azimuth,polarity,surface,series,id,order_series,annotations
0,2.236362,9.289332,7.069177,0.415299,0.742150,0.612666,0.009900,0.064134,0.239981,15.131408,8.775031,0.248601,foo,foo7,3,3,"${\bf{x}}_{\beta \,{\bf{3}},0}$"
1,3.746931,5.376924,3.602518,0.494110,0.538028,0.431800,1.293404,3.921952,12.104145,18.838738,18.251817,12.789250,foo5,boo,2,2,"${\bf{x}}_{\beta \,{\bf{2}},0}$"
2,1.009549,0.824145,7.769260,0.351293,0.300496,0.649191,0.635225,3.700102,13.204787,15.870903,9.741438,13.728097,lala,boo,1,2,"${\bf{x}}_{\beta \,{\bf{1}},0}$"
3,7.664131,0.616635,7.039326,0.698482,0.289669,0.611109,0.034972,0.271297,10.147020,1.544194,7.345296,10.150706,foo5,boo,2,2,"${\bf{x}}_{\beta \,{\bf{2}},1}$"
4,4.119740,2.983007,6.754145,0.513561,0.413130,0.596230,0.019351,0.621533,2.959713,11.865248,1.783266,3.024331,lala,boo,1,2,"${\bf{x}}_{\beta \,{\bf{1}},1}$"
5,7.300673,3.089303,9.703372,0.679520,0.418676,0.750100,0.240818,0.719070,5.363631,8.047284,18.515804,5.416973,feeeee,foo20,4,5,"${\bf{x}}_{\beta \,{\bf{4}},0}$"


### Additional Data

In [72]:
ad = gp.AdditionalData(surface_points, orientations, grid, faults, surfaces, rescaling)


In [73]:
ad

values
Structure isLith                                                                    True
          isFault                                                                   True
          number faults                                                                1
          number surfaces                                                              4
          number series                                                                3
          number surfaces per series                                           [2, 1, 1]
          len surfaces surface_points                                       [2, 2, 1, 1]
          len series surface_points                                            [4, 1, 1]
          len series orientations                                              [4, 1, 1]
Options   dtype                                                                  float64
          output                                                                 geology
          theano_optimizer                                                  fast_compile
          device                                                                     cpu
          verbosity                                                                   []
Kriging   range                                                                  1732.05
          $C_o$                                                                  71428.6
          drift equations                                                      [3, 0, 0]
          nugget grad                                                               0.01
          nugget scalar                                                            1e-06
Rescaling rescaling factor                                                        19.167
          centers                      [3.8617381861045827, 4.649959731166183, 4.9116...

In [74]:
ad.structure_data

,values
isLith,True
isFault,True
number faults,1
number surfaces,4
number series,3
number surfaces per series,"[2, 1, 1]"
len surfaces surface_points,"[2, 2, 1, 1]"
len series surface_points,"[4, 1, 1]"
len series orientations,"[4, 1, 1]"


In [75]:
ad.options

,values
dtype,float64
output,geology
theano_optimizer,fast_compile
device,cpu
verbosity,[]


In [76]:
ad.options.df

,dtype,output,theano_optimizer,device,verbosity
values,float64,geology,fast_compile,cpu,[]


In [77]:
ad.options.df.dtypes

dtype               category
output              category
theano_optimizer    category
device              category
verbosity             object
dtype: object

In [78]:
ad.kriging_data

,values
range,1732.05
$C_o$,71428.6
drift equations,"[3, 0, 0]"
nugget grad,0.01
nugget scalar,1e-06


In [79]:
ad.rescaling_data

,values
rescaling factor,19.167
centers,"[3.8617381861045827, 4.649959731166183, 4.9116..."


### Interpolator

In [80]:
interp = gp.Interpolator(surface_points, orientations, grid, surfaces, faults, ad)

AttributeError: module 'gempy' has no attribute 'Interpolator'

In [ ]:
interp.compile_th_fn()

In [ ]:
interp.print_theano_shared()